In [66]:
import os
import pandas as pd
import numpy as np
import h5py

def read_and_combine(cwd):
    all_file_names = os.listdir(cwd)
    df = pd.DataFrame()
    for i in all_file_names:
        sub_dir = os.path.join(cwd, i)
        if os.path.isdir(sub_dir):
            sub_dir_filenames = os.listdir(sub_dir)
            sub_df = sub_read_and_combine(sub_dir, sub_dir_filenames)
            df = pd.concat([df,sub_df])
    return df

def sub_read_and_combine(sub_dir, sub_dir_filenames):
    sub_df = pd.DataFrame()
    for j in sub_dir_filenames:
        j = os.path.join(sub_dir, j)
        if j[-3:] == 'csv':
            temp_df = pd.read_csv(j)
            sub_df = pd.concat([sub_df,temp_df])
    return sub_df


# df = read_and_combine("d:/stock/rawdata/index_stock_weight")
# df.sort_values(['Enddt','Stkcd'],inplace=True)
# df['Stkcd'] = df['Stkcd'].astype(str).str.zfill(6)
# df = df.loc[df['Stkcd'].str[:2].isin(['00','60'])]
# stocks = df['Stkcd'].unique()
# stocks = pd.Series(stocks)
# stocks.to_csv('stocks.txt',index=False,header=None)

In [67]:
price = read_and_combine("d:/stock/rawdata/price")
value = read_and_combine("d:/stock/rawdata/value")
price = pd.merge(price, value, on=['Symbol', 'TradingDate'], how='left')
del value
st = read_and_combine("d:/stock/rawdata/st")
st.columns = ['Symbol', 'TradingDate', 'Markettype', 'Trdsta']
price = pd.merge(price, st, on=['Symbol', 'TradingDate'], how='left')
del st
hk = read_and_combine("d:/stock/rawdata/hk")
hk = hk.loc[hk['MarketLinkCode'].isin(['HKEXtoSZSE', 'HKEXtoSSE'])]
hk.drop(['MarketLinkCode','ShareholdingRatioA'],axis=1,inplace=True)
price = pd.merge(price, hk, on=['Symbol', 'TradingDate'], how='left')
del hk
price.sort_values(['TradingDate', 'Symbol'],inplace=True)
price.reset_index(drop=True,inplace=True)

In [68]:
price = price.loc[((price['Trdsta']==1) & (price['Filling']==0) & (price['StateCode']==0) & (price['Volume'] != 0))]
price.drop(['Trdsta', 'Filling', 'StateCode', 'Markettype'],axis=1,inplace=True)
price.head()

,TradingDate,Symbol,OpenPrice,ClosePrice,HighPrice,LowPrice,Volume,Amount,Distance,AvgPrice,TurnoverRate1,MarketValue,CirculatedMarketValue,PE2TTM,PBV2A,ShareholdingRatioB
1112,2008-01-02,1,1436.056,1416.660,1445.008,1404.351,20052473,7.634657e+08,3.0,1420.129,0.00874,8.710360e+10,6.672404e+10,32.870487,6.697154,NaN
1113,2008-01-02,2,1561.118,1570.263,1587.478,1527.228,93879671,2.719854e+09,3.0,1558.536,0.01366,1.922862e+11,1.874193e+11,41.408777,6.851200,NaN
1115,2008-01-02,6,259.752,264.509,266.392,255.788,3500865,9.237655e+07,3.0,261.506,0.01381,6.768361e+09,5.834511e+09,26.261303,3.933732,NaN
1118,2008-01-02,12,220.972,224.376,225.923,217.670,4640052,1.003130e+08,3.0,223.024,0.00391,2.031523e+10,1.263774e+10,59.882058,6.448242,NaN
1119,2008-01-02,16,48.648,50.288,50.288,47.992,10671937,9.632590e+07,3.0,49.337,0.01773,4.692901e+09,3.598984e+09,26.473781,1.561033,NaN


In [69]:
ind = pd.read_csv("d:/stock/rawdata/ind/STK_INDUSTRYCLASS.csv")

ind = ind.loc[(ind['IndustryClassificationID']== 'P0212') | (ind['IndustryClassificationID']== 'P0217')]
ind =ind[['Symbol','ImplementDate','IndustryCode','IndustryName']]
zx_ind_map = pd.read_csv("d:/stock/rawdata/zx_ind.csv")
zx_ind_map2 = zx_ind_map.loc[(zx_ind_map['INDCLASSIFYSYSTEMCODE']=='P0212') & (zx_ind_map['RANK']==2)]
zx_ind_map1 = zx_ind_map.loc[(zx_ind_map['INDCLASSIFYSYSTEMCODE']=='P0212') & (zx_ind_map['RANK']==1)]
zx_ind_map3 = zx_ind_map.loc[(zx_ind_map['INDCLASSIFYSYSTEMCODE']=='P0212') & (zx_ind_map['RANK']==3)]
zx_ind_map2.drop(['INDCLASSIFYSYSTEMCODE','RANK'],axis=1,inplace=True)
zx_ind_map2.columns = ['id2','ind2','id1']
zx_ind_map1.drop(['INDCLASSIFYSYSTEMCODE','RANK','P_INDUSTRYCODE'],axis=1,inplace=True)
zx_ind_map1.columns = ['id1','ind1']
zx_ind_map3.drop(['INDCLASSIFYSYSTEMCODE','RANK'],axis=1,inplace=True)
zx_ind_map3.columns = ['id3','ind3','id2']
indf1 = pd.merge(zx_ind_map3,zx_ind_map2,on='id2',how='left')
indf1 = pd.merge(indf1,zx_ind_map1,on='id1',how='left')

zx_ind_map2 = zx_ind_map.loc[(zx_ind_map['INDCLASSIFYSYSTEMCODE']=='P0217') & (zx_ind_map['RANK']==2)]
zx_ind_map1 = zx_ind_map.loc[(zx_ind_map['INDCLASSIFYSYSTEMCODE']=='P0217') & (zx_ind_map['RANK']==1)]
zx_ind_map3 = zx_ind_map.loc[(zx_ind_map['INDCLASSIFYSYSTEMCODE']=='P0217') & (zx_ind_map['RANK']==3)]
zx_ind_map2.drop(['INDCLASSIFYSYSTEMCODE','RANK'],axis=1,inplace=True)
zx_ind_map2.columns = ['id2','ind2','id1']
zx_ind_map1.drop(['INDCLASSIFYSYSTEMCODE','RANK','P_INDUSTRYCODE'],axis=1,inplace=True)
zx_ind_map1.columns = ['id1','ind1']
zx_ind_map3.drop(['INDCLASSIFYSYSTEMCODE','RANK'],axis=1,inplace=True)
zx_ind_map3.columns = ['id3','ind3','id2']
indf2 = pd.merge(zx_ind_map3,zx_ind_map2,on='id2',how='left')
indf2 = pd.merge(indf2,zx_ind_map1,on='id1',how='left')

indf = pd.concat([indf1,indf2])
ind = pd.merge(ind,indf,left_on=['IndustryCode'],right_on=['id3'],how='left')
ind = ind[['Symbol','ImplementDate','ind1']]

d:\Conda\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [70]:
ind.columns=['Symbol', 'TradingDate', 'Industry']
price = pd.merge(price, ind, on=['Symbol','TradingDate'],how='outer')
price.sort_values(['Symbol','TradingDate'],inplace=True)
price['Industry'] = price.groupby('Symbol')['Industry'].ffill()
price['Industry'] = price.groupby('Symbol')['Industry'].bfill()
price.dropna(subset = ['Volume'],inplace=True)
price.sort_values(['Symbol','TradingDate'],inplace=True)
price.reset_index(drop=True,inplace=True)

,TradingDate,Symbol,OpenPrice,ClosePrice,HighPrice,LowPrice,Volume,Amount,Distance,AvgPrice,TurnoverRate1,MarketValue,CirculatedMarketValue,PE2TTM,PBV2A,ShareholdingRatioB,Industry
4621733,2022-10-13,605499,131.932,139.878,147.071,131.932,1477655.0,208508460.0,1.0,143.758,0.00369,5.492137e+10,1.043617e+10,43.185191,12.958883,0.0167,食品饮料
4623214,2022-10-14,605499,140.866,140.642,144.666,137.066,966912.0,133772177.0,1.0,140.948,0.00242,5.522138e+10,1.049318e+10,43.421089,13.029671,0.0169,食品饮料
4624693,2022-10-17,605499,140.347,145.685,146.704,137.759,953442.0,134749815.0,1.0,143.984,0.00238,5.720143e+10,1.086943e+10,44.978021,13.496870,0.0178,食品饮料
4626172,2022-10-18,605499,147.101,149.954,154.447,146.704,770332.0,114594093.0,1.0,151.552,0.00193,5.887747e+10,1.118791e+10,46.295908,13.892338,0.0191,食品饮料
4627653,2022-10-19,605499,151.492,144.728,151.564,144.157,484482.0,69713379.0,1.0,146.595,0.00121,5.682542e+10,1.079798e+10,44.682361,13.408149,NaN,食品饮料


In [76]:
price = price.loc[price['TradingDate'].str[:4] >= '2013']
price.head()

,TradingDate,Symbol,OpenPrice,ClosePrice,HighPrice,LowPrice,Volume,Amount,Distance,AvgPrice,TurnoverRate1,MarketValue,CirculatedMarketValue,PE2TTM,PBV2A,ShareholdingRatioB,Industry
1381736,2013-01-04,1,799.630,783.461,805.999,780.031,44385137.0,717567547.0,4.0,792.133,0.00866,8.192237e+10,4.965467e+10,6.112378,0.966080,NaN,银行
1383037,2013-01-07,1,782.971,798.650,801.100,778.071,35716925.0,578450488.0,1.0,793.505,0.00697,8.351061e+10,5.061734e+10,6.230879,0.984809,NaN,银行
1384332,2013-01-08,1,798.650,783.951,802.080,777.091,31247912.0,501360094.0,1.0,786.156,0.00610,8.197361e+10,4.968573e+10,6.116201,0.966684,NaN,银行
1385626,2013-01-09,1,781.991,777.091,784.931,774.151,25132915.0,399696183.0,1.0,779.198,0.00491,8.125634e+10,4.925098e+10,6.062684,0.958225,NaN,银行
1386922,2013-01-10,1,777.091,777.581,788.850,773.661,24003027.0,383347733.0,1.0,782.530,0.00469,8.130757e+10,4.928203e+10,6.066506,0.958830,NaN,银行


In [72]:
# price['time'] = price['time'].str.replace('-', '', regex=False).astype(int) # transfer string time to int time
# rawdata = price.values
# colnames=np.array(price.columns)
# colnames = np.array([[i.encode('utf8')]
#                     for i in colnames])  # 需要先转成utf8格式才能写入h5
# dt = h5py.special_dtype(vlen=str)
# with h5py.File('rawdata.h5', 'w') as f:
#     f.create_dataset('rawdata', data=rawdata, compression='gzip', compression_opts=9)
#     f.create_dataset('data_cols', data=colnames, dtype=dt, compression='gzip', compression_opts=9)